# Sea ice export

## Import libraries

In [ ]:
import xarray as xr
from c3s_eqc_automatic_quality_control import download

## Define parameters

In [ ]:
year_start = 2002
year_stop = 2020

## Define request

In [ ]:
time_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": ["01", "02", "03", "04", "10", "11", "12"],
}

requests = {
    "satellite-sea-ice-drift": time_request
    | {
        "variable": "all",
        "region": "northern_hemisphere",
        "day": [f"{day:02d}" for day in range(1, 32)],
        "version": "1_0",
    },
    "satellite-sea-ice-thickness": time_request
    | {
        "processing_level": "level_3",
        "satellite_mission": ["envisat", "cryosat_2"],
        "variable": ["sea_ice_thickness"],
        "temporal_resolution": ["monthly"],
        "version": "3_0",
    },
    "satellite-sea-ice-concentration": time_request
    | {
        "variable": "all",
        "version": "3_0",
        "sensor": "ssmis",
        "region": ["northern_hemisphere"],
        "cdr_type": ["cdr"],
        "temporal_aggregation": "daily",
        "day": [f"{day:02d}" for day in range(1, 32)],
    },
}

## Define functions to cache

In [ ]:
def regrid_and_resample(ds, coarsen, time_resample):
    if coarsen:
        ds = ds.coarsen(xc=3, yc=3).mean(keep_attrs=True)
    if time_resample:
        grouped = ds.resample(time="1MS")
        ds = grouped.mean(keep_attrs=True).dropna("time", how="all")
        sum_vars = set(ds.data_vars) & {"dX", "dY"}
        if sum_vars:
            ds_sum = grouped.sum(keep_attrs=True).dropna("time", how="all")
            ds = ds.merge(
                ds_sum[sum_vars].rename({var: f"{var}_sum" for var in sum_vars})
            )
    return ds

## Download and transform

In [ ]:
datasets = {}
for collection_id, request in requests.items():
    print(f"{collection_id = }")
    datasets[collection_id] = download.download_and_transform(
        collection_id,
        request,
        chunks={"year": 1},
        transform_func=regrid_and_resample,
        transform_func_kwargs={
            "coarsen": collection_id != "satellite-sea-ice-drift",
            "time_resample": collection_id != "satellite-sea-ice-thickness",
        },
    )
common_times = xr.merge([ds[["time"]] for ds in datasets.values()], join="inner")
datasets = {
    collection_id: ds.sel(time=common_times["time"])
    for collection_id, ds in datasets.items()
}